In [11]:
import torch
import src.modules
import math

## Generate Dataset

In [46]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    
    target = abs(input.pow(2).sum(1).sub(1 / (2*math.pi)).sign().add(-1).div(2).long())
    return input, target

In [49]:
N = 1000
train_input, train_target = generate_disc_set(N)
test_input, test_target = generate_disc_set(N)